In [1]:
# %pip install -q -U bitsandbytes huggingface_hub[hf_xet]

In [ ]:
import os
import time
from dataclasses import dataclass

import numpy as np
import pandas as pd
import torch
from datasets import Dataset
from google.colab import userdata
from huggingface_hub import login
from peft import LoraConfig, TaskType, get_peft_model
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    EvalPrediction,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
)

In [ ]:
@dataclass
class Config:
    data_dir: str = "/content"
    output_dir: str = "/content"

    categories_path: str = os.path.join(data_dir, "categories.txt")
    train_path: str = os.path.join(data_dir, "train.csv")
    test_path: str = os.path.join(data_dir, "test.csv")
    labeled_train_path: str = os.path.join(output_dir, "labeled_train.csv")
    augmented_train_path: str = os.path.join(output_dir, "augmented_train.csv")
    submission_path: str = os.path.join(output_dir, "submission.csv")
    model_path: str = os.path.join(output_dir, "model")

    device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    hf_token: str = userdata.get("HF_TOKEN")


config = Config()
login(config.hf_token)

In [5]:
def load_data() -> tuple[pd.DataFrame, pd.DataFrame, list[str]]:
    """Загружает данные из файлов

    Returns:
        tuple[pd.DataFrame, pd.DataFrame, list[str]]: train_df, test_df, categories
    """
    train_df = pd.read_csv(config.train_path)
    test_df = pd.read_csv(config.test_path)

    with open(config.categories_path) as f:
        categories = [line.strip() for line in f if line.strip()]

    return train_df, test_df, categories


train_df, test_df, categories = load_data()

In [ ]:
system_prompt = """Ты — ассистент по классификации отзывов с маркетплейсов.
Твоя задача — определить категорию товара из списка: бытовая техника, обувь, одежда, посуда, текстиль, товары для детей, украшения и аксессуары, электроника, нет товара.

Правила:
- Верни только одну категорию из списка, без пояснений, в нижнем регистре.
- Если нельзя определить товар — верни: нет товара.
- При неявном описании ориентируйся на функции, характеристики, материалы и другие признаки.
- Игнорируй упоминания доставки, упаковки, цены, акций, продавца, приложения, возвратов.
- Не путай одежду и обувь.
- Не путай электронику и бытовую технику.
"""


def normalize_label(raw_label: str) -> str:
    """Нормализует категорию: приводит к нижнему регистру и убирает лишние символы"""
    text = raw_label.lower().strip()
    if text in categories:
        return text
    return "нет товара"


def label_review(review: str, llm: tuple[AutoModelForCausalLM, AutoTokenizer]) -> str:
    """Классифицирует отзыв по категории с помощью локальной LLM"""
    model, tokenizer = llm

    formatted_system_prompt = {"role": "system", "content": system_prompt}
    formatted_review = {"role": "user", "content": review}
    messages = [formatted_system_prompt, formatted_review]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)
    input_len = inputs["input_ids"].shape[-1]

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            do_sample=False,
            temperature=None,
            top_p=None,
            top_k=None,
        )
        outputs = outputs[0][input_len:]

    decoded = tokenizer.decode(outputs, skip_special_tokens=True)
    label = normalize_label(decoded)
    return label


def label_dataset(dataset: pd.DataFrame, llm: tuple[AutoModelForCausalLM, AutoTokenizer]) -> pd.DataFrame:
    """Размечает датасет с помощью LLM."""
    labeled_dataset = dataset.copy()

    if os.path.exists(config.labeled_train_path):
        print(f"Найден сохраненный прогресс: {config.labeled_train_path}")
        labeled_dataset = pd.read_csv(config.labeled_train_path)
        start_idx = labeled_dataset["label"].notna().sum()
    else:
        labeled_dataset["label"] = None
        start_idx = 0

    for i in tqdm(range(start_idx, len(labeled_dataset)), desc="Разметка датасета"):
        review = labeled_dataset.loc[i, "text"]
        labeled_dataset.loc[i, "label"] = label_review(review, llm)

        if (i + 1) % 100 == 0 or (i + 1) == len(labeled_dataset):
            labeled_dataset.to_csv(config.labeled_train_path, index=False)
            print(f"Сохранен прогресс до шага {i + 1}")
    return labeled_dataset


def setup_llm() -> tuple[AutoModelForCausalLM, AutoTokenizer]:
    """Загружает локальную LLM и процессор"""
    model_name = "google/gemma-3-12b-it"
    quantization_config = BitsAndBytesConfig(load_in_4bit=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float32,
        quantization_config=quantization_config,
        device_map=config.device,
    )
    return model, tokenizer

In [7]:
if os.path.exists(config.labeled_train_path):
    print("Используем сохраненную разметку")
    labeled_train_df = pd.read_csv(config.labeled_train_path)
    start_idx = labeled_train_df["label"].notna().sum()
    print(f"Найдено размеченных отзывов: {start_idx}/{len(labeled_train_df)}")
    if start_idx < len(labeled_train_df):
        llm = setup_llm()
        print("LLM загружена, продолжаем разметку")
        labeled_train_df = label_dataset(train_df, llm, config.labeled_train_path)
else:
    print("Сохраненная разметка не найдена, размечаем данные")
    llm = setup_llm()
    print("LLM загружена")
    labeled_train_df = label_dataset(train_df, llm, config.labeled_train_path)

print(rf"Размеченный датасет сохранен по пути: {config.labeled_train_path}")
print(f"Кол-во отзывов в размеченном датасете: {len(labeled_train_df)}")


Используем сохраненную разметку
Найдено размеченных отзывов: 1818/1818
Размеченный датасет сохранен по пути: /content/labeled_train.csv
Кол-во отзывов в размеченном датасете: 1818


In [8]:
def print_label_distribution(df: pd.DataFrame, categories: list[str], title: str) -> None:
    print(title)
    total = len(df)
    for label in categories:
        count = df[df["label"] == label].shape[0]
        percentage = (count / total * 100) if total else 0.0
        print(f"- {label}: {count} ({percentage:.1f}%)")


print_label_distribution(labeled_train_df, categories, "Распределение меток после разметки:")

Распределение меток после разметки:
- бытовая техника: 0 (0.0%)
- обувь: 128 (7.0%)
- одежда: 1148 (63.1%)
- посуда: 2 (0.1%)
- текстиль: 56 (3.1%)
- товары для детей: 3 (0.2%)
- украшения и аксессуары: 7 (0.4%)
- электроника: 0 (0.0%)
- нет товара: 474 (26.1%)


In [9]:
generate_synthetic_data_prompt = """Сгенерируй {n_samples} реалистичных отзывов покупателей на русском языке о товарах категории: {label}.

Требования:
- Верни только отзывы, без пояснений и комментариев.
- Разделяй отзывы символом "|".
- Каждый отзыв - 1–2 предложения.
- Стиль - разговорный. Допускаются сленг, сокращения, лёгкие ошибки, эмоции, восклицания.
- В каждом отзыве должно быть понятно, о каком товаре идёт речь, через описание функций, характеристик, материалов, применения и других признаков.
- Не упоминай название категории или товара.
- Разнообразие: отзывы должны отличаться по стилю, длине и лексике. Используй разные точки зрения (молодёжь, взрослые, родители, пожилые).
- Тональность: положительные, нейтральные и негативные отзывы в случайной пропорции.
- Избегай повторов отзывов и шаблонных формулировок."""


def generate_synthetic_data(
    label: str,
    samples_per_class: int,
    llm: tuple[AutoModelForCausalLM, AutoTokenizer],
) -> list[str]:
    """Генерирует ~samples_per_class синтетических отзывов под заданную категорию"""
    model, tokenizer = llm

    all_reviews = []
    n_samples = 10
    while len(all_reviews) < samples_per_class:
        print(f"{label}: {len(all_reviews)} сгенерировано")
        prompt = generate_synthetic_data_prompt.format(label=label, n_samples=n_samples)
        formatted_prompt = [{"role": "user", "content": prompt}]

        inputs = tokenizer.apply_chat_template(
            formatted_prompt,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt",
        ).to(model.device)
        input_len = inputs["input_ids"].shape[-1]

        with torch.inference_mode():
            generated = model.generate(
                **inputs,
                do_sample=True,
                temperature=1.2,
                max_new_tokens=100_000,
            )
            generated = generated[0][input_len:]
            decoded = tokenizer.decode(generated, skip_special_tokens=True).strip()

        reviews = [text.strip() for text in decoded.split("|") if text.strip()][1:]
        all_reviews.extend(reviews)
        n_samples += 10

    return all_reviews[:samples_per_class]


def balance_with_synthetic_data(
    labeled_df: pd.DataFrame,
    llm: tuple[AutoModelForCausalLM, AutoTokenizer],
    samples_per_class: int = 250,
) -> pd.DataFrame:
    """Добалансирует датасет синтетическими примерами с помощью локальной LLM"""
    current_counts = labeled_df["label"].value_counts().to_dict()
    need_samples = {label: samples_per_class - current_counts.get(label, 0) for label in categories}

    synthetic_data = []
    for label, n_samples in need_samples.items():
        if n_samples <= 0:
            continue
        data = generate_synthetic_data(label, n_samples, llm)
        synthetic_data.extend([{"text": text, "label": label} for text in data])

    synthetic_dataset = pd.DataFrame(synthetic_data)
    augmented_dataset = pd.concat([labeled_df, synthetic_dataset], axis=0, ignore_index=True)

    return augmented_dataset

In [10]:
if os.path.exists(config.augmented_train_path):
    print("Используем сохраненный аугментированный датасет")
    augmented_train_df = pd.read_csv(config.augmented_train_path)
else:
    print("Сохраненный аугментированный датасет не найден, генерируем синтетические данные")
    if not llm:
        llm = setup_llm()
    augmented_train_df = balance_with_synthetic_data(labeled_train_df, llm, samples_per_class=400)
    augmented_train_df.to_csv(config.augmented_train_path, index=False)
    print(rf"Аугментированный датасет сохранен по пути: {config.augmented_train_path}")
print(f"Кол-во отзывов в аугментированном датасете: {len(augmented_train_df)}")
print_label_distribution(augmented_train_df, categories, "Распределение меток после добавления синтетики:")


Используем сохраненный аугментированный датасет
Кол-во отзывов в аугментированном датасете: 4422
Распределение меток после добавления синтетики:
- бытовая техника: 400 (9.0%)
- обувь: 400 (9.0%)
- одежда: 1148 (26.0%)
- посуда: 400 (9.0%)
- текстиль: 400 (9.0%)
- товары для детей: 400 (9.0%)
- украшения и аксессуары: 400 (9.0%)
- электроника: 400 (9.0%)
- нет товара: 474 (10.7%)


In [ ]:
def print_text_stats(dataset: pd.DataFrame, label: str) -> None:
    print(label)
    text_lengths = dataset["text"].str.len()
    average_length = text_lengths.mean()
    print(f"Средняя длина текста: {average_length:.1f} символов")
    print(f"Минимальная длина: {text_lengths.min()} символов")
    print(f"Максимальная длина: {text_lengths.max()} символов")
    print(f"Медианная длина: {text_lengths.median():.1f} символов")


print_text_stats(train_df, "Статистики текстов оригинального датасета:")
print()
print_text_stats(augmented_train_df, "Статистики текстов аугментированного датасета:")

Статистики текстов оригинального датасета:
Средняя длина текста: 91.0 символов
Минимальная длина: 2 символов
Максимальная длина: 228 символов
Медианная длина: 79.0 символов

Статистики текстов аугментированного датасета:
Средняя длина текста: 79.0 символов
Минимальная длина: 2 символов
Максимальная длина: 228 символов
Медианная длина: 72.0 символов


In [12]:
def setup_bert_model() -> tuple[RobertaForSequenceClassification, AutoTokenizer]:
    """Загружает модель для классификации ai-forever/ruRoberta-large и конфигурирует LoRA"""
    num_labels = len(categories)
    model_name = "ai-forever/ruRoberta-large"

    model = RobertaForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
    )

    for name, param in model.named_parameters():
        if name.startswith("classifier"):
            param.requires_grad = True
        else:
            param.requires_grad = False

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer


bert_model, bert_tokenizer = setup_bert_model()

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

In [13]:
def tokenize_batch(batch: dict[str, list[str]]) -> dict[str, list[int]]:
    """Токенизация текста в батче"""
    return bert_tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=256,
    )


def prepare_dataset(
    dataset: pd.DataFrame, categories: list[str]
) -> tuple[Dataset, Dataset, dict[str, int], dict[int, str]]:
    """Подготовка датасетов: label encoding, разделение на train и validation, токенизация"""
    label2id = {label: idx for idx, label in enumerate(categories)}
    id2label = {idx: label for label, idx in label2id.items()}
    dataset = dataset.copy()
    dataset["label"] = dataset["label"].map(label2id)

    stratify_column = dataset["label"] if dataset["label"].nunique() > 1 else None
    train_df, val_df = train_test_split(
        dataset,
        test_size=0.1,
        random_state=42,
        shuffle=True,
        stratify=stratify_column,
    )

    train_dataset = Dataset.from_pandas(train_df[["text", "label"]].reset_index(drop=True)).map(
        tokenize_batch, batched=True
    )
    eval_dataset = Dataset.from_pandas(val_df[["text", "label"]].reset_index(drop=True)).map(
        tokenize_batch, batched=True
    )

    return train_dataset, eval_dataset, label2id, id2label


train_dataset, eval_dataset, label2id, id2label = prepare_dataset(augmented_train_df, categories)

Map:   0%|          | 0/3979 [00:00<?, ? examples/s]

Map:   0%|          | 0/443 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(eval_pred: EvalPrediction) -> dict[str, float]:
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    weighted_f1 = f1_score(labels, predictions, average="weighted")
    return {"weighted_f1": weighted_f1}


lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.10,
    target_modules=["query", "value"],
    bias="none",
    task_type=TaskType.SEQ_CLS,
    modules_to_save=["classifier"],
)

lora_bert_model = get_peft_model(bert_model, lora_config)

args = TrainingArguments(
    output_dir=config.model_path,
    eval_strategy="steps",
    eval_steps=25,
    save_strategy="steps",
    save_steps=25,
    save_total_limit=5,
    load_best_model_at_end=True,
    metric_for_best_model="weighted_f1",
    greater_is_better=True,
    num_train_epochs=7,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_ratio=0.15,
    label_smoothing_factor=0.05,
    # fp16=True,
    logging_strategy="steps",
    logging_steps=25,
    dataloader_num_workers=2,
    report_to="none",
    seed=42,
    save_safetensors=True,
)

trainer = Trainer(
    model=lora_bert_model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [16]:
trainer.train()

Step,Training Loss,Validation Loss,Weighted F1
25,2.114600,1.885059,0.175707
50,1.540200,0.941632,0.759079
75,0.819000,0.687134,0.852496
100,0.629400,0.613625,0.876591
125,0.565900,0.588969,0.883468
150,0.514300,0.576249,0.881780
175,0.508800,0.548488,0.912211
200,0.471900,0.557101,0.906904
225,0.461300,0.535562,0.918016
250,0.469000,0.540258,0.909223


TrainOutput(global_step=441, training_loss=0.6367614480102954, metrics={'train_runtime': 3026.6338, 'train_samples_per_second': 9.203, 'train_steps_per_second': 0.146, 'total_flos': 1.3091438559919104e+16, 'train_loss': 0.6367614480102954, 'epoch': 7.0})

In [18]:
def submit() -> pd.DataFrame:
    start_time = time.time()

    test_dataset = Dataset.from_pandas(test_df[["text"]].reset_index(drop=True)).map(tokenize_batch, batched=True)

    inference_start = time.time()
    predictions = trainer.predict(test_dataset)
    inference_time = time.time() - inference_start

    pred_ids = np.argmax(predictions.predictions, axis=1)
    pred_labels = [id2label[idx] for idx in pred_ids]

    submission = pd.DataFrame({"category": pred_labels})
    submission.to_csv(config.submission_path, index=False)
    print(rf"Сабмит сохранен по пути: {config.submission_path}")

    total_time = time.time() - start_time

    inference_avg_time = inference_time / len(test_df)
    overall_avg_time = total_time / len(test_df)

    print(f"Среднее время на пример (только inference): {inference_avg_time:.4f} сек.")
    print(f"Среднее время на пример (всего pipeline): {overall_avg_time:.4f} сек.")

    return submission


In [19]:
submission = submit()

Map:   0%|          | 0/7276 [00:00<?, ? examples/s]

Сабмит сохранен по пути: /content/submission.csv
Среднее время на пример (только inference): 0.0455 сек.
Среднее время на пример (всего pipeline): 0.0458 сек.
